<a href="https://colab.research.google.com/github/damlakaynarca/Fashion-Mn-st/blob/main/Fashion_Mn%C4%B1st_%C3%96zellikler_Hangi_S%C4%B1n%C4%B1fta_%2B%C3%96zellik_%C3%87%C4%B1kar%2BS%C4%B1n%C4%B1fla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Fashion MNIST veri setini yükle
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Verileri 0-1 aralığında normalize et

# Verileri 4D tensöre çevir (CNN için)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Etiketleri one-hot encoding yap
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# CNN modelini oluştur
input_layer = Input(shape=(28, 28, 1))
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(128, kernel_size=(3, 3), activation='relu')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
flatten = Flatten()(pool3)
dense = Dense(128, activation='relu')(flatten)
output_layer = Dense(10, activation='softmax')(dense)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model.fit(x_train, y_train_cat, epochs=5, batch_size=128, validation_split=0.1)

# Özellik çıkarımı (3. katmandan)
feature_extractor = Model(inputs=model.inputs, outputs=pool3)
train_features = feature_extractor.predict(x_train)
test_features = feature_extractor.predict(x_test)

# Özellikleri düzleştir
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

# Verileri ölçeklendir
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(train_features_flat)
x_test_scaled = scaler.transform(test_features_flat)

# Eğitim ve test verilerini ayır
x_train_scaled, x_val_scaled, y_train, y_val = train_test_split(x_train_scaled, y_train, test_size=0.1, random_state=42)

# Sınıflandırıcıları oluştur ve eğit
svm_model = SVC()
svm_model.fit(x_train_scaled, y_train)

knn_model = KNeighborsClassifier()
knn_model.fit(x_train_scaled, y_train)

nb_model = GaussianNB()
nb_model.fit(x_train_scaled, y_train)

dt_model = DecisionTreeClassifier()
dt_model.fit(x_train_scaled, y_train)

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(x_train_scaled, y_train)

lda_model = LinearDiscriminantAnalysis()
lda_model.fit(x_train_scaled, y_train)

# Doğruluk hesapla
svm_accuracy = svm_model.score(x_val_scaled, y_val)
knn_accuracy = knn_model.score(x_val_scaled, y_val)
nb_accuracy = nb_model.score(x_val_scaled, y_val)
dt_accuracy = dt_model.score(x_val_scaled, y_val)
lr_accuracy = lr_model.score(x_val_scaled, y_val)
lda_accuracy = lda_model.score(x_val_scaled, y_val)

# Tahminleri al ve gerçek etiketlerle karşılaştırarak hangi sınıfa ait olduğunu göster
svm_predictions = svm_model.predict(x_val_scaled)
knn_predictions = knn_model.predict(x_val_scaled)
nb_predictions = nb_model.predict(x_val_scaled)
dt_predictions = dt_model.predict(x_val_scaled)
lr_predictions = lr_model.predict(x_val_scaled)
lda_predictions = lda_model.predict(x_val_scaled)

# Tahmin edilen sınıfları gerçek etiketlerle eşleştirerek göster
# Tahmin edilen sınıfları gerçek etiketlerle eşleştirerek göster
for idx, (svm_pred, knn_pred, nb_pred, dt_pred, lr_pred, lda_pred, true_label) in enumerate(zip(svm_predictions, knn_predictions, nb_predictions, dt_predictions, lr_predictions, lda_predictions, y_val)):
    print("Veri", idx + 1, "SVM Tahmini:", svm_pred, "| KNN Tahmini:", knn_pred, "| Naive Bayes Tahmini:", nb_pred, "| Decision Tree Tahmini:", dt_pred, "| Lojistik Regresyon Tahmini:", lr_pred, "| LDA Tahmini:", lda_pred, "| Gerçek Etiket:", true_label)


Görüntülenen çıkış son 5000 satıra kısaltıldı.
Veri 1001 SVM Tahmini: 1 | KNN Tahmini: 1 | Naive Bayes Tahmini: 1 | Decision Tree Tahmini: 1 | Lojistik Regresyon Tahmini: 1 | LDA Tahmini: 1 | Gerçek Etiket: 1
Veri 1002 SVM Tahmini: 2 | KNN Tahmini: 4 | Naive Bayes Tahmini: 2 | Decision Tree Tahmini: 4 | Lojistik Regresyon Tahmini: 2 | LDA Tahmini: 2 | Gerçek Etiket: 2
Veri 1003 SVM Tahmini: 2 | KNN Tahmini: 2 | Naive Bayes Tahmini: 2 | Decision Tree Tahmini: 2 | Lojistik Regresyon Tahmini: 2 | LDA Tahmini: 2 | Gerçek Etiket: 2
Veri 1004 SVM Tahmini: 6 | KNN Tahmini: 6 | Naive Bayes Tahmini: 4 | Decision Tree Tahmini: 6 | Lojistik Regresyon Tahmini: 6 | LDA Tahmini: 6 | Gerçek Etiket: 2
Veri 1005 SVM Tahmini: 2 | KNN Tahmini: 0 | Naive Bayes Tahmini: 2 | Decision Tree Tahmini: 2 | Lojistik Regresyon Tahmini: 2 | LDA Tahmini: 2 | Gerçek Etiket: 0
Veri 1006 SVM Tahmini: 5 | KNN Tahmini: 5 | Naive Bayes Tahmini: 5 | Decision Tree Tahmini: 5 | Lojistik Regresyon Tahmini: 5 | LDA Tahmini: 5 